In [1]:
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

import spacy
from spacy.matcher import Matcher

from thefuzz import fuzz, process

In [9]:
import sys
sys.setrecursionlimit(10**6)

In [2]:
#df_matched = pd.read_csv('matched_data.csv')
df1 = pd.read_csv('source_1.csv')
df2 = pd.read_csv('source_2.csv')

print(f'Shape of source_1 : {df1.shape}')
print(f'Shape of source_2 : {df2.shape}')

Shape of source_1 : (13238, 2)
Shape of source_2 : (48943, 2)


In [3]:
# list of items to iterate over. 
# Will be modified in every iteration where a match is found
#src1 = df1.name.to_list()
#src2 = df2.name.to_list()

# name(key), id(value) dictionaries
src1_dict = df1.name.to_dict()
src1_dict = dict([(value, key) for key, value in src1_dict.items()])

src2_dict = df2.name.to_dict()
src2_dict = dict([(value, key) for key, value in src2_dict.items()])
    
    

In [4]:
# lists to create pandas dataframes
matched_ids = []
matched_names_ids = []
def closest_matches(src1:list, src2:list, names_id_src1:dict, names_id_src2:dict):
    if len(src1) == 0:
        return matched_ids, matched_names_ids
    for item in src1:
        best_match = process.extractOne(item, src2, scorer=fuzz.partial_token_sort_ratio)
        confidence = best_match[1]
        if confidence == 100:
            src1_item_id = names_id_src1[item]

            src2_item = best_match[0]
            src2_item_id = names_id_src2[src2_item]

            matched_ids.append(dict(source_1=src1_item_id, source_2=src2_item_id))
            df_ids = pd.DataFrame(matched_ids)
            df_ids.to_csv('matched_ids.csv')

            matched_names_ids.append(dict(id1=src1_item_id, source_1=item, id2=src2_item_id,source_2=src2_item))
            df_names_ids = pd.DataFrame(matched_names_ids)
            df_names_ids.to_csv('matched_names_ids.csv')

            src1.remove(item)
            src2.remove(src2_item)

            print(df_names_ids)
            print()
            return closest_matches(src1, src2, names_id_src1, names_id_src2)
        else:
            src1.remove(item)
            return closest_matches(src1, src2, names_id_src1, names_id_src2)
    

In [ ]:
ids_matched, names_ids_matched = closest_matches(src1, src2, src1_dict, src2_dict)

       id1                                           source_1    id2  \
0        0  Horses, asses, mules and hinnies; live, pure-b...  20744   
1        1           Horses; live, pure-bred breeding animals  29816   
2        2  Horses; live, other than pure-bred breeding an...  36008   
3        3  Horses, asses, mules and hinnies; live, other ...  35887   
4        5           Cattle; live, pure-bred breeding animals   2890   
...    ...                                                ...    ...   
1127  2923  Oth.organic derivat.of hydrazine or of hydroxy...  17864   
1128  2937                                           Terbacil  14948   
1129  2939                                    Propythiouracil  42537   
1130  2948      Prostaglandins, thromboxanes and leukotrienes  19433   
1131  2952               Other hormones, prostaglandins, etc.  39301   

            source_2  
0            hinnies  
1            animals  
2           breeding  
3              mules  
4             cattle

In [6]:
len(src1)

10289

In [7]:
len(src2)

47812

In [8]:
len(matched_ids)

1131

In [10]:
len(src1_dict)

12582